# 감성분석 -> 머신러닝
- 데이터셋: 전처리
- BoW 모델:
    - 단어를 특성 벡터로 변환
    - tf-idf 단어 적합성 평가
    - 텍스트 데이터 정제
    - 문서를 토큰으로 나누기
- LogisticRegression

In [7]:
import pandas as pd
from glob import glob

file_lists = glob('C://python_src//pandas-data-analysis//machine_learning//data//train//neg//*.txt')
pd_lists = []

for file_path in file_lists[:500]:
    with open(file_path,'r',encoding='utf-8') as f:
        data = {
            'review' : f.read(),
            'target' : 0
        }

    df = pd.DataFrame([data])
    pd_lists.append(df)

train_neg_df = pd.concat(pd_lists, ignore_index=True)
train_neg_df.head()

,review,target
0,Story of a man who has unnatural feelings for ...,0
1,Airport '77 starts as a brand new luxury 747 p...,0
2,This film lacked something I couldn't put my f...,0
3,"Sorry everyone,,, I know this is supposed to b...",0
4,When I was little my parents took me along to ...,0


In [8]:
import pandas as pd
from glob import glob

file_lists = glob('C://python_src//pandas-data-analysis//machine_learning//data//train//pos//*.txt')
pd_lists = []

for file_path in file_lists[:500]:
    with open(file_path,'r',encoding='utf-8') as f:
        data = {
            'review' : f.read(),
            'target' : 1
        }

    df = pd.DataFrame([data])
    pd_lists.append(df)

train_pos_df = pd.concat(pd_lists, ignore_index=True)
train_pos_df.head()

,review,target
0,Bromwell High is a cartoon comedy. It ran at t...,1
1,Homelessness (or Houselessness as George Carli...,1
2,Brilliant over-acting by Lesley Ann Warren. Be...,1
3,This is easily the most underrated film inn th...,1
4,This is not the typical Mel Brooks film. It wa...,1


In [9]:
train_df = pd.concat( [train_pos_df, train_neg_df], ignore_index=True )
train_df.head()

,review,target
0,Bromwell High is a cartoon comedy. It ran at t...,1
1,Homelessness (or Houselessness as George Carli...,1
2,Brilliant over-acting by Lesley Ann Warren. Be...,1
3,This is easily the most underrated film inn th...,1
4,This is not the typical Mel Brooks film. It wa...,1


In [ ]:
train_df.to_csv("movie_data.csv", index=False, encoding='utf-8')

In [ ]:
df = pd.read_csv('movie_data.csv')
df.head()

## BOW(Bag of Words) 모델
<!-- https://www.ibm.com/kr-ko/think/topics/bag-of-words -->
- 문자를 숫자벡터 
- 단어의 등장 횟수를 카운트
---
- 전체 훈련데이터에서 모든 고유한 단어(토큰)로 어휘 사전을 만들음
- 각 문서(review data)를 사전을 기준으로 벡터화 -> n번째 단어가 문서에 3번 나오면, 벡터의 n번째 값이 3이 됨.
> - 문서1: "나는 영화가 좋다." <br>
> - 문서2: "나는 영화가 싫다." <br>
> 사전: ('나는':0 '영화가':1 '좋다':2 '싫다':3) <br>
- 벡터화는 사전의 크기만큼 모든 문장의 길이를 동일하게 함
> - 문서 1벡터: 토큰 [0,1,2] <br> -> 벡터화[1,1,1,0]
> - 문서 2벡터: 토큰 [0,1,3] <br> -> [1,1,0,1]

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
docs = np.array([
    "The sun is shining",
    "The weather is sweet"
])

bag = count.fit_transform(docs)

In [ ]:
count.vocabulary_

## tf(t,d)
- 문서 d에 등장한 단어 t의 횟수를 tf(t,d)
- Bow를 보완하면서 좀 더 정교한 텍스트 벡터화 방식 TF-IDF(Term Frequency - Inverse Document Frequency)
    - TF: 특정 문서에서 자주 등장하는 단어
    - IDF: 전체 문서에서 드물게 등장하는 단어
- 특정 문서에서 자주 등장하지만, 전체 문장에서 드물게 등장하는 단어에 높은 가중치 부여 -> 그 문장을 잘 대표하는 핵심 단어를 찾는다
- `TF(t,d)`: `단어 t가 문장 d에 나타난 횟수 / 문서 d의 모든 단어 수` 
- `IDF(t,D)`: `log( 총 문서 수[D] / 단어 t를 포함한 문서 수 df(t ) )` (why? _log_ 단어의 희귀성을 너무 과하게 반영하지 않도록 스케일링) 
- 분모에 +1(사이킷런의 경우): 분모가 0이 되는 것 방지
- `log(1+[D]/1+df(f))`
> `TF-IDF(t,d,D) = TF(t,d) x IDF(t,D)` 

---


- 예시 <br>
        "나는"
    - TF: 리뷰에 3번 나옴 (가중치 높음)
    - IDF: 전체 10,000개 리뷰 중 9,000개 문장에서 나옴 (매우 낮음)
    - TF-IDF: 높음 x 매우낮음 = 낮음(중요도가 낮다) <br>
    <br>
    
    "명작"
    - TF: 리뷰에 2번 나옴 (가중치 높음)
    - IDF: 전체 10,000개 리뷰 중 50개 문장에서 나옴 (매우 높음)
    - TF-IDF: 높음 x 매우높음 = 높음(핵심 단어)


In [ ]:
# 데이터 정제... html tag 와 같은 불필요한 string이 보임... 특수기호 기타등등.  < - ' 
import re
def preprocessor(s):
    # 1. 영문, 공백, ., , 만 남기기
    clean = re.sub(r'[^A-Za-z\s.,]+', '', s)
    # 2. 연속된 마침표(...)를 마침표 하나로
    clean = re.sub(r'\.{2,}', '.', clean)
    # 3. 연속된 공백 정리
    clean = re.sub(r'\s+', ' ', clean).strip()
    return clean

In [ ]:
df['review'] = df.review.apply(preprocessor)

### 불용어(Stopwords)
- 자연어 처리 시 분석에 크게 기여하지 않는, 의미가 거의 없거나 중요하지 않은 단어
-  'the', 'a', 'an', 'is', 'in', 'and' ...

In [ ]:
# 불용어
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords') #불용어 사전 다운로드
stops = stopwords.words('english')

In [ ]:
df.head() #전처리 완료, 정규식을 이용한??

### 어간추출 (Stemming)
- 단어의 접사(suffix, prefix)를 잘라서 원형(어간)으로 통일하는 과정
- PorterStemmer, LancasterStemmer 등 규칙 기반 알고리즘

In [ ]:
from nltk.stem.porter import PorterStemmer

def tokenizer(text):
    return text.split()

porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

# 어간 추출 Stemming: 단어의 접미사(-s,-es,-ing ...)를 강제로 제거 -> '단어의 원형' 찾기
tokenizer(df.review[0][:100])

In [ ]:
tokenizer('runners like running')

##### 실행

In [ ]:
from sklearn.model_selection import train_test_split

X = df.review
y = df.target

X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape, y_train.shape

In [ ]:
# 어간추출 -> stopwords에 포함된 단어 제거
porter = PorterStemmer()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split() if word not in stops]

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [ ]:
tfidf = TfidfVectorizer(
    tokenizer=tokenizer_porter,
    ngram_range=(1,1) #(1,1) 유니그램(unigram, 단일단어)만 사용)
)

pipe = Pipeline([
    ('tfidf', tfidf),
    ('clf', LogisticRegression())

])

In [ ]:
pipe.fit(X_train,y_train)

In [ ]:
# 원본 데이터 로드

# 토크나이저 함수 정의
    # 텍스트 전처리
    # 공백을 기준으로, 단어단위로 분리
    # 영어는 전부 소문자로 변환
    # 어간 추출
    # 불용어 제거

# TFIDF를 정의
    # 토크나이저 매개변수 = 토크나이저 함수
    # ngram=(1,1)

# 파이프라인으로 tfidf, 머신러닝

# 파이프라인으로 학습

# 파이프라인으로 평가(calssification_report)

# 과적합 여부 확인
    # train data와 test data로 성능 비교

# train 폴더의 데이터로 학습 후, test폴더의 데이터로 평가하기

In [5]:
import pandas as pd
from glob import glob

file_lists = glob('C://python_src//pandas-data-analysis//machine_learning//data//test//neg//*.txt')
pd_lists = []

for file_path in file_lists[:500]:
    with open(file_path,'r',encoding='utf-8') as f:
        data = {
            'review' : f.read(),
            'target' : 0
        }

    train_df = pd.DataFrame([data])
    pd_lists.append(train_df)

test_neg_df = pd.concat(pd_lists, ignore_index=True)

file_lists = glob('C://python_src//pandas-data-analysis//machine_learning//data//test//pos//*.txt')
pd_lists = []

for file_path in file_lists[:500]:
    with open(file_path,'r',encoding='utf-8') as f:
        data = {
            'review' : f.read(),
            'target' : 1
        }

    test_df = pd.DataFrame([data])
    pd_lists.append(test_df)

test_pos_df = pd.concat(pd_lists, ignore_index=True)
test_pos_df.head()

,review,target
0,I went and saw this movie last night after bei...,1
1,Actor turned director Bill Paxton follows up h...,1
2,As a recreational golfer with some knowledge o...,1
3,"I saw this film in a sneak preview, and it is ...",1
4,Bill Paxton has taken the true story of the 19...,1


In [11]:
train_df = pd.concat( [train_pos_df, train_neg_df], ignore_index=True )
test_df = pd.concat( [test_pos_df, test_neg_df], ignore_index=True )

In [59]:
train_df.head()

,review,target
0,Bromwell High is a cartoon comedy. It ran at t...,1
1,Homelessness (or Houselessness as George Carli...,1
2,Brilliant over-acting by Lesley Ann Warren. Be...,1
3,This is easily the most underrated film inn th...,1
4,This is not the typical Mel Brooks film. It wa...,1


In [17]:
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
import re

porter = PorterStemmer()


def custom_tokenizer(text):
    # 전처리
    # 1. 영문, 공백, ., , 만 남기기
    clean = re.sub(r'[^A-Za-z\s.,]+', '', text)
    # 2. 연속된 마침표(...)를 마침표 하나로
    clean = re.sub(r'\.{2,}', '.', clean)
    # 3. 연속된 공백 정리
    clean = re.sub(r'\s+', ' ', clean).strip()
    # 단어분리-어간분리-불용어제거
    return [porter.stem(word) for word in clean.split() if word not in stops]


tfidf = TfidfVectorizer(
    tokenizer=custom_tokenizer,
    ngram_range=(1,1),
    token_pattern=None
)


pipe = Pipeline([
    ('tfid',tfidf),
    ('clf',LogisticRegression())
])

In [18]:
train_sample = train_df.sample(200, random_state=42)
test_sample = test_df.sample(200, random_state=42)

In [19]:
# 샘플로 테스트

from sklearn.metrics import classification_report

X_train = train_sample['review']
y_train = train_sample['target']
X_test  = test_sample['review']
y_test  = test_sample['target']


pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)

print(classification_report(y_test, y_pred, zero_division=0))

NameError: name 'stops' is not defined

In [ ]:
# 원본 데이터

# X_train = train_df['review']
# X_test  = test_df['target']
# y_train = train_df['review']
# y_test  = test_df['target']

# pipe.fit(X_train, y_train)
# y_pred = pipe.predict(X_test)
print(classification_report(y_test, y_pred, zero_division=0))

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index=pipe.classes_, columns=pipe.classes_)

plt.figure(figsize=(5,4))
sns.heatmap(cm_df, annot=True, fmt='d', cmap='Blues')
plt.title("Confusion Matrix (Sample 200)")
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

NameError: name 'y_test' is not defined